# Introduction

In this notebook, we will explore the OpenAI library. We will also acquire data in different ways to inform our queries.

The setup is not different from what we have done before.

In [18]:
%load_ext dotenv
%dotenv ../../05_src/.env
%dotenv ../../05_src/.secrets

In [19]:
import sys
sys.path.append('../../05_src/')

In [20]:
from utils.logger import get_logger
_logs = get_logger(__name__)

In [21]:
from openai import OpenAI
import os
client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                api_key='any value',
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})

2. We create an API call and store the result in the variable `response`. Notice that the call specifies the model that we want to use, as well as an input. This is a simple call; the [responses API can handle more complex calls](https://platform.openai.com/docs/api-reference/responses).

# Longer Context

Thus far, we have created our context and prompt manually. We can use any of Python's text manipulation tools to create documents. 

For instance, in the code below, we will use the requests library to download a book from Project Gutenberg and include it in the context. Some useful links are:

- Python's [request](https://pypi.org/project/requests/) library.
- Python's request library [documentation](https://requests.readthedocs.io/en/latest/).

In [22]:
import requests
file_url = 'https://www.gutenberg.org/cache/epub/223/pg223.txt'
book = requests.get(file_url)

The code below shows how to retrieve the response headers. 

In [23]:
dict(book.headers) #explore it by converting into dictionary and exploring headers
#will tell me what I got from my http request

{'date': 'Wed, 11 Feb 2026 06:41:32 GMT',
 'server': 'Apache',
 'last-modified': 'Sun, 01 Feb 2026 09:42:33 GMT',
 'accept-ranges': 'bytes',
 'content-length': '434868',
 'x-backend': 'gutenweb6',
 'content-type': 'text/plain; charset=utf-8'}

We can also obtain the payload using `book.text` or `book.content`.

In [24]:
print(book.text) #can read what we downloaded

The Project Gutenberg eBook of The wisdom of Father Brown
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.

Title: The wisdom of Father Brown

Author: G. K. Chesterton

Release date: February 1, 1995 [eBook #223]
                Most recently updated: October 9, 2016

Language: English

Credits: Produced by Martin Ward, and David Widger


*** START OF THE PROJECT GUTENBERG EBOOK THE WISDOM OF FATHER BROWN ***




Produced by Martin Ward





THE WISDOM OF FATHER BROWN

By G. K. Chesterton



To

LUCIAN OLDERSHAW



CONTENTS


     1.  The Absence of Mr Glass
     2.  The Paradise of Thieves
   

With added content, we can also write more interesting prompts. Notice the use of Python's f-strings or formatted strings. 

When using formatted strings, remember two things:

+ Formatted strings are prefixed with f:  `f'This is a formatted string: {some_variable}.`
+ You can enclose variable names in curly brackes, `{...}`, and their values will appear in formatted strings: `f'The variable value is {variable}.`

In [ ]:
prompt = f"""
    You are a literary historian. 
    Given the following context from a book, do the following:
    
    1. Identify the book's title and author.
    2. Determine how many stories are included in the book.
    3. Summarize concisely in no more than 5 paragraphs the first story of the book called "The Absence of Mr Glass".
        
    The book is the following: 
    <book> 
    {book.text}
    </book>

    Provide your response in the following format:
    Title: <title>
    Author: <author>
    Number of Stories: <number_of_stories>
    Summary: <summary>
"""

#use <> </> tags (XML-like tagging) to help the model understand where the book content is

In [26]:
prompt

'\n    You are a literary historian. \n    Given the following context from a book, do the following:\n\n    1. Identify the book\'s title and author.\n    2. Determine how many stories are included in the book.\n    3. Summarize concisely in no more than 5 paragraphs the first story of the book called "The Absence of Mr Glass".\n\n    The book is the following: \n    <book> \n    \ufeffThe Project Gutenberg eBook of The wisdom of Father Brown\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\r\n\r\nTitle: The wisdom of Father Brown\r\n\r\nAuthor: G. K. Chesterton\r\n\r\n

In [27]:
response = client.responses.create(
    model = 'gpt-4o',
    input = prompt,
)


Taking a while because we are using gpt 4o model, so more complexity will generally take longer. Also giving it a large amount of text

We use `Ipython` to format the output using markdown and create a friendlier display. (Since we are using Jupyter notebooks)

In [28]:
from IPython.display import display, Markdown

display(Markdown(response.output_text))

Title: The Wisdom of Father Brown  
Author: G. K. Chesterton  
Number of Stories: 12  

Summary:  

"The Absence of Mr Glass" begins with Father Brown visiting Scarborough, where he encounters Dr. Orion Hood, a well-known criminologist. Dr. Hood's consulting rooms are an epitome of precision, with everything in perfect order. Father Brown arrives, appearing slightly out of place, and is mistaken for someone in need of Dr. Hood’s criminological expertise. Brown mentions the MacNab family and a romantic entanglement involving Maggie MacNab and a lodger named Todhunter.  

Father Brown's interest in the case leads him and Dr. Hood to investigate. Maggie reports that Todhunter has been murdered, and Mr. Glass is suspected. They find Todhunter bound, yet unharmed, in his room. Dr. Hood deduces it’s a blackmail case where Todhunter was the victim, but Father Brown's instincts tell him something else.

Father Brown realizes Todhunter staged the entire scene, including his own fake ties and the supposed conflict with Mr. Glass. Todhunter, aspiring to be a professional conjurer, was merely practicing his tricks involving ropes and ventriloquism. The mismatched hat and broken glass were parts of his act, not evidence of crime.

The story concludes with Father Brown unveiling the innocent nature of Todhunter’s activities, demonstrating the importance of understanding subtler, human motives over mere deductive reasoning.

# Adding a Developer Prompt

Using the OpenAI library, we can specify prompts for different roles:

+ *Developer* messages are instructions provided by the application developer, prioritized ahead of user messages. In previous iterations of the API, these were called *System* messages.
+ *User* messages are instructions provided by an end user, prioritized below developer messages.
+ *Assistant* messages are generated by the model.

In [29]:
system_prompt = "You are a specialist in ancient fables who speaks like Bugs Bunny."

#used to be called system prompt, now called developer prompt
#it is component of prompt inserted before the user prompt (above it)
#Is used to set the behavior of the model

In [30]:
the_ant = """
Ants were once men and made their living by tilling the soil. But, not content with the results of their own work, they were always casting longing eyes upon the crops and fruits of their neighbours, which they stole, whenever they got the chance, and added to their own store. At last their covetousness made Jupiter so angry that he changed them into Ants. But, though their forms were changed, their nature remained the same: and so, to this day, they go about among the cornfields and gather the fruits of others’ labour, and store them up for their own use.
"""

prompt = f"""
    Please, extract the main message of this fable written by Aesop.
    The fable is the following: 
    <fable>
    {the_ant}
    </fable>
"""

In [ ]:
#After buidling prompt, send them off to internet - to the API - and get a response back
response = client.responses.create(
    model="gpt-4o",
    instructions = system_prompt, #instruction argument will take system prompt and embed it at the top of your message to your model
    input = prompt,
)



In [32]:
from IPython.display import display, Markdown

display(Markdown(response.output_text))

Eh, what's up, Doc? So, in this fable, Aesop's givin' us a message all about greed and envy, see? Those fellas, originally men, couldn't keep their mitts off their neighbors' crops. Ol' Jupiter gets fed up with their sticky fingers and turns 'em into ants. But even as ants, they can't shake their thievin' ways. The moral of the story? If ya let greed take over, it might end up changin' you permanently, doc!